<br></br>

<div align="center">

<h1 align="center">
    Cuaderno para el trabajo de clasificación relacional
    <br></br>
    Evaluar modelos
</h1>

<h6 align="center">
    Antonio Macías Ferrera (antmacfer1@alum.us.es)
    <br></br>
    Delfín Santana Rubio (delsanrub@alum.us.es)
    <br></br>
    Universidad de Sevilla
</h6>

<br></br>

## **Control de Versiones**
    
| **Fecha**  | **Versión** | **Descripción**               |
| :--------- | :---------- | :---------------------------- |
| 27/05/2024 | v1r0        | Primera versión del cuaderno. |
| 29/05/2024 | v1r1        | Inclsuión de otras medidas de centralidad.          |
| 31/05/2024 | v1r2        | Correcciones a algunos métodos e inclusión de otros.|
| 01/06/2024 | v1r2        | Eliminación de métricas no óptimas.                 |
| 07/06/2024 | v1r3        | Correcciones de formato.      |


</div>

<br></br>

## **Índice de contenido**

1. [Introducción](#introducción)
2. [Instanciación del grafo](#creacion)
3. [Evaluación de las métricas](#evaluacion)
    - [Árboles de decisión CART](#cart)
    - [Naive Bayes](#naive-bayes)
    - [KNN](#knn)
    - [Random Forest](#random-forest)
    - [Gradient Boosting](#gradient-boosting)

<br></br>

# <a name="introducción"></a> 1. **Introducción**

En este cuaderno se encuentra todo el codigo neceario para evaluar los modelos de aprendizaje automático con las distintas métricas seleccionadas anteriormente.

A continuación se importan todas las librerías y métodos necesarios para la ejecución del código.

<br></br>

In [1]:
# Pandas
import pandas as pd

# Numpy
import numpy as np

# Codificadores de sci-kit learn
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# Train-test split, búsqueda en rejilla, validaciones cruzadas y selección de modelos
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score

# Modelos a entrenar (CART, Naive Bayes, Knn, Support Vector Machines)
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Discretizador para Naive Bayes
from sklearn.preprocessing import KBinsDiscretizer

# Normalizador para Knn
from sklearn.preprocessing import MinMaxScaler

# Tubería
from sklearn.pipeline import Pipeline

# <a name="creacion"></a> 2. **Instanciación del grafo**

Aquí se cargan los datos del csv en un dataframe de Pandas, y se determinan cuales serán los atributos de entrenamiento (las métricas seleccionadas) y el atributo objetivo (tipos de grupos de Facebook).
<br></br>

In [2]:
values = pd.read_csv('dict.csv')
values.head()

,greedy_modularity_communities_id,label_propagation_communities_id,betweenness_centrality,degree_centrality,k_path_centrality,closeness_centrality,harmonic_centrality,square_clustering,clustering,target
0,5,0,0.000000,0.000045,0.000003,0.182482,4224.448882,0.000000,0.000000,tvshow
1,2,1,0.000153,0.001513,0.000100,0.257752,6230.390079,0.130612,0.481283,government
2,4,5,0.000003,0.000534,0.000035,0.189540,4419.497403,0.347089,0.651515,company
3,22,6,0.000052,0.000445,0.000029,0.219007,5147.241270,0.160836,0.511111,government
4,3,8,0.000125,0.002270,0.000149,0.230387,5493.355556,0.176843,0.420392,politician


In [3]:
atributos_continuos = ["greedy_modularity_communities_id", "label_propagation_communities_id", "betweenness_centrality", "degree_centrality", "k_path_centrality", "closeness_centrality", "harmonic_centrality", "clustering", "square_clustering"]
atributos = values.loc[:, atributos_continuos]
atributos.head()

,greedy_modularity_communities_id,label_propagation_communities_id,betweenness_centrality,degree_centrality,k_path_centrality,closeness_centrality,harmonic_centrality,clustering,square_clustering
0,5,0,0.000000,0.000045,0.000003,0.182482,4224.448882,0.000000,0.000000
1,2,1,0.000153,0.001513,0.000100,0.257752,6230.390079,0.481283,0.130612
2,4,5,0.000003,0.000534,0.000035,0.189540,4419.497403,0.651515,0.347089
3,22,6,0.000052,0.000445,0.000029,0.219007,5147.241270,0.511111,0.160836
4,3,8,0.000125,0.002270,0.000149,0.230387,5493.355556,0.420392,0.176843


In [4]:
objetivo = values['target']
objetivo.head()

0        tvshow
1    government
2       company
3    government
4    politician
Name: target, dtype: object

In [5]:
codificador_objetivo = LabelEncoder()
# El método fit_transform ajusta el codificador a los datos y, a continuación,
# codifica estos adecuadamente. En este caso no necesitamos mantener el
# atributo objetivo como una Series de Pandas.
objetivo = codificador_objetivo.fit_transform(objetivo)
print(f'Clases detectadas: {codificador_objetivo.classes_}')
objetivo

Clases detectadas: ['company' 'government' 'politician' 'tvshow']


array([3, 1, 0, ..., 1, 0, 3])

# <a name="evaluacion"></a> 3. **Evaluación de las métricas**

En este apartado se realizará una búsqueda en rejilla con validación cruzada para cada modelo de aprendizaje automático seleccionado (CART, Naive Bayes, KNN, Random Forest, Gradient Boosting). Para cada búsqueda en rejilla, se comprueba la importancia de cada atributo de entrenamiento y se obtiene la precisión del modelo, después se entrena el modelo con los atributos con mayor importancia y se comprueba si la precisión del modelo entrenado con los mejores atributos es mejor que el modelo entrenado con todos los atributos.
<br></br>

## <a name="cart"></a> **Árboles de decisión CART**

In [6]:
# Establecer división entre atributos de prueba y de entrenamiento 
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos, objetivo, 
    test_size=.2, 
    stratify=objetivo)

In [7]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, árbol de decisión CART)
rejilla = {
    'max_depth': [3, 5, 8, 12, None],
    'min_samples_split': [2, 10, 40, 50, 100, 200],
    'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
    'criterion': ['gini', 'entropy']
}

modelo = DecisionTreeClassifier()

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(estimator=modelo, param_grid=rejilla, cv=5, scoring='accuracy', n_jobs=-1)

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 8, 12, None],
                         'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
                         'min_samples_split': [2, 10, 40, 50, 100, 200]},
             scoring='accuracy')

In [8]:
# Mejor modelo obtenido
mejor_modelo_CART = validaciones_cruzadas.best_estimator_
mejor_modelo_CART

DecisionTreeClassifier(criterion='entropy')

In [9]:
validaciones_cruzadas.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [10]:
# Evaluar la importancia de los atributos
importancias_CART = mejor_modelo_CART.feature_importances_

# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_CART.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"- Accuracy: {accuracy}")

# Crear un DataFrame para visualizar las importancias
if importancias_CART is not None:
    importancias_atributos = pd.DataFrame({'Atributo': atributos_entrenamiento.columns, 'Importancia': importancias_CART})
    importancias_atributos = importancias_atributos.sort_values(by='Importancia', ascending=False)
    print(importancias_atributos)

# Seleccionar los atributos más importantes (umbral puede ser ajustado)
selector = SelectFromModel(mejor_modelo_CART, prefit=True, threshold='mean')
atributos_seleccionado = selector.transform(atributos_entrenamiento)

print("- Atributos seleccionados:")
print(atributos_entrenamiento.columns[selector.get_support()])

- Accuracy: 0.7979528259902091
                           Atributo  Importancia
1  label_propagation_communities_id     0.365781
0  greedy_modularity_communities_id     0.245804
8                 square_clustering     0.081952
6               harmonic_centrality     0.077224
5              closeness_centrality     0.072135
7                        clustering     0.060879
2            betweenness_centrality     0.051013
3                 degree_centrality     0.024901
4                 k_path_centrality     0.020311
- Atributos seleccionados:
Index(['greedy_modularity_communities_id', 'label_propagation_communities_id'], dtype='object')


/opt/homebrew/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [11]:
# Entrenar y evaluar el modelo con las mejores características
atributos_entrenamiento_selected = selector.fit_transform(atributos_entrenamiento, objetivo_entrenamiento)
atributos_prueba_selected = selector.transform(atributos_prueba)
mejor_modelo_CART.fit(atributos_entrenamiento_selected, objetivo_entrenamiento)
predicciones_selected = mejor_modelo_CART.predict(atributos_prueba_selected)
accuracy_selected = accuracy_score(objetivo_prueba, predicciones_selected)

print(f"- Accuracy con mejores características: {accuracy_selected}")

- Accuracy con mejores características: 0.8460169114374722


## <a name="naive-bayes"></a> **Naive Bayes**

In [12]:
# Discretizar los atributos para poder usar Naive Bayes
discretizador = KBinsDiscretizer(
    n_bins=4,  # Cada atributo se discretiza en 4 intervalos
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='quantile'  # Cada intervalo contiene la misma cantidad de datos
)

atributos_discretizados = atributos.copy()
atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)

/opt/homebrew/lib/python3.12/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


In [13]:
# Establecer división entre atributos de prueba y de entrenamiento 
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos_discretizados, objetivo, 
    test_size=.2, 
    stratify=objetivo)

In [14]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
rejilla = {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]}

modelo = CategoricalNB()

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(estimator=modelo, param_grid=rejilla, cv=10, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=10, estimator=CategoricalNB(),
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]},
             scoring='accuracy')

In [15]:
# Mejor modelo obtenido
mejor_modelo_NB = validaciones_cruzadas.best_estimator_
mejor_modelo_NB

CategoricalNB()

In [16]:
validaciones_cruzadas.best_params_

{'alpha': 1.0}

In [17]:
# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_NB.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"- Accuracy: {accuracy}")

# Evaluar la importancia de los atributos manualmente
baseline_score = mejor_modelo_NB.score(atributos_prueba, objetivo_prueba)
importance_scores = []

for i in atributos_discretizados.columns:
    X_train_dropped = atributos_entrenamiento[[i]]
    X_test_dropped = atributos_prueba[[i]]
    
    model_dropped = CategoricalNB()
    model_dropped.fit(X_train_dropped, objetivo_entrenamiento)
    dropped_score = model_dropped.score(X_test_dropped, objetivo_prueba)
    importance_score = baseline_score - dropped_score
    importance_scores.append(importance_score)

# Crear un DataFrame para visualizar las importancias
importancias_NB = pd.DataFrame({
    'Atributo': atributos_discretizados.columns,
    'Importancia': importance_scores
})
importancias_NB.sort_values(by='Importancia', ascending=False, inplace=True)
print(importancias_NB)

- Accuracy: 0.4668446817979528
                           Atributo  Importancia
2            betweenness_centrality     0.133066
0  greedy_modularity_communities_id     0.083667
7                        clustering     0.077882
8                 square_clustering     0.067868
3                 degree_centrality     0.060525
4                 k_path_centrality     0.059635
5              closeness_centrality     0.035826
6               harmonic_centrality     0.031153
1  label_propagation_communities_id     0.016911


In [18]:
# Seleccionar los atributos más importantes (K=3 para mejor resultado)
selector = SelectKBest(score_func=f_classif, k=3)
atributos_seleccionados = selector.fit_transform(atributos_entrenamiento, objetivo_entrenamiento)

print("- Atributos seleccionados:")
print(atributos_entrenamiento.columns[selector.get_support()])

- Atributos seleccionados:
Index(['label_propagation_communities_id', 'closeness_centrality',
       'harmonic_centrality'],
      dtype='object')


In [19]:
# Entrenar y evaluar el modelo con las mejores características
atributos_entrenamiento_selected = selector.fit_transform(atributos_entrenamiento, objetivo_entrenamiento)
atributos_prueba_selected = selector.transform(atributos_prueba)
mejor_modelo_NB.fit(atributos_entrenamiento_selected, objetivo_entrenamiento)
predicciones_selected = mejor_modelo_NB.predict(atributos_prueba_selected)
accuracy_selected = accuracy_score(objetivo_prueba, predicciones_selected)

print(f"- Accuracy con mejores características: {accuracy_selected}")

- Accuracy con mejores características: 0.48531375166889185


## <a name="knn"></a> **KNN**

In [20]:
# Establecer división entre atributos de prueba y de entrenamiento 
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos_discretizados, objetivo, 
    test_size=.2, 
    stratify=objetivo)

In [21]:
# Normalizar atributos
normalizador = MinMaxScaler(feature_range=(0,1))

# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
tuberia = Pipeline([('normalizador', normalizador),
                   ('knn', KNeighborsClassifier())])

rejilla = {
    'knn__n_neighbors': [1,3,5,7],
    'knn__metric': ['euclidean', 'manhattan']
}

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(tuberia, rejilla, cv=5, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('normalizador', MinMaxScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': [1, 3, 5, 7]},
             scoring='accuracy')

In [22]:
# Mejor modelo obtenido
mejor_modelo_KNN = validaciones_cruzadas.best_estimator_
mejor_modelo_KNN

Pipeline(steps=[('normalizador', MinMaxScaler()),
                ('knn',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=7))])

In [23]:
validaciones_cruzadas.best_params_

{'knn__metric': 'euclidean', 'knn__n_neighbors': 7}

In [24]:
# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_KNN.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"- Accuracy: {accuracy}")

# Evaluar la importancia de los atributos manualmente
baseline_score = mejor_modelo_KNN.score(atributos_prueba, objetivo_prueba)
importance_scores = []

for i in atributos_discretizados.columns:
    X_train_dropped = atributos_entrenamiento[[i]]
    X_test_dropped = atributos_prueba[[i]]
    
    model_dropped = KNeighborsClassifier()
    model_dropped.fit(X_train_dropped, objetivo_entrenamiento)
    dropped_score = model_dropped.score(X_test_dropped, objetivo_prueba)
    importance_score = baseline_score - dropped_score
    importance_scores.append(importance_score)

# Crear un DataFrame para visualizar las importancias
importancias_KNN = pd.DataFrame({
    'Atributo': atributos_discretizados.columns,
    'Importancia': importance_scores
})
importancias_KNN.sort_values(by='Importancia', ascending=False, inplace=True)
print(importancias_KNN)

- Accuracy: 0.5587449933244326
                           Atributo  Importancia
2            betweenness_centrality     0.262350
8                 square_clustering     0.256787
0  greedy_modularity_communities_id     0.255452
6               harmonic_centrality     0.238095
1  label_propagation_communities_id     0.214508
3                 degree_centrality     0.187138
7                        clustering     0.186026
5              closeness_centrality     0.154428
4                 k_path_centrality     0.150868


In [25]:
# Seleccionar los atributos más importantes (K=3 para mejor resultado)
selector = SelectKBest(score_func=f_classif)
atributos_seleccionados = selector.fit_transform(atributos_entrenamiento, objetivo_entrenamiento)

print("- Atributos seleccionados:")
print(atributos_entrenamiento.columns[selector.get_support()])

- Atributos seleccionados:
Index(['greedy_modularity_communities_id', 'label_propagation_communities_id',
       'betweenness_centrality', 'degree_centrality', 'k_path_centrality',
       'closeness_centrality', 'harmonic_centrality', 'clustering',
       'square_clustering'],
      dtype='object')


/opt/homebrew/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:776: UserWarning: k=10 is greater than n_features=9. All the features will be returned.
  warnings.warn(


In [26]:
# Entrenar y evaluar el modelo con las mejores características
atributos_entrenamiento_selected = selector.fit_transform(atributos_entrenamiento, objetivo_entrenamiento)
atributos_prueba_selected = selector.transform(atributos_prueba)
mejor_modelo_KNN.fit(atributos_entrenamiento_selected, objetivo_entrenamiento)
predicciones_selected = mejor_modelo_KNN.predict(atributos_prueba_selected)
accuracy_selected = accuracy_score(objetivo_prueba, predicciones_selected)

print(f"- Accuracy con mejores características: {accuracy_selected}")

- Accuracy con mejores características: 0.5587449933244326


/opt/homebrew/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:776: UserWarning: k=10 is greater than n_features=9. All the features will be returned.
  warnings.warn(


En este caso, con Knn, al probar con cualquier combinacion de características obteníamos menos accuracy en comparación con usar todas las cinco.

## <a name="random-forest"></a> **Random forest**

In [28]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
modelo = RandomForestClassifier()

rejilla = {
    'n_estimators': [100, 200, 300],
    'max_features': [None, 'sqrt', 'log2'],
}

rejilla = {
    'max_depth': [3, 5, 8, 12, None],
    'min_samples_split': [2, 10, 40, 50, 100, 200],
    'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
    'criterion': ['gini', 'entropy']
}

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(modelo, rejilla, cv=5, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 8, 12, None],
                         'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
                         'min_samples_split': [2, 10, 40, 50, 100, 200]},
             scoring='accuracy')

In [29]:
# Mejor modelo obtenido
mejor_modelo_RFC = validaciones_cruzadas.best_estimator_
mejor_modelo_RFC

RandomForestClassifier(max_depth=12, min_samples_split=10)

In [30]:
validaciones_cruzadas.best_params_

{'criterion': 'gini',
 'max_depth': 12,
 'min_samples_leaf': 1,
 'min_samples_split': 10}

In [31]:
# Evaluar la importancia de los atributos
importancias_RFC = mejor_modelo_RFC.feature_importances_

# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_RFC.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"Accuracy: {accuracy}")

# Crear un DataFrame para visualizar las importancias
if importancias_RFC is not None:
    importancias_atributos = pd.DataFrame({'Atributo': atributos_entrenamiento.columns, 'Importancia': importancias_RFC})
    importancias_atributos = importancias_atributos.sort_values(by='Importancia', ascending=False)
    print(importancias_atributos)

# Seleccionar los atributos más importantes (umbral puede ser ajustado)
selector = SelectFromModel(mejor_modelo_RFC, prefit=True, threshold='mean')
atributos_seleccionado = selector.transform(atributos_entrenamiento)

print("Atributos seleccionados:")
print(atributos_entrenamiento.columns[selector.get_support()])

Accuracy: 0.5925678682688028
                           Atributo  Importancia
1  label_propagation_communities_id     0.217267
6               harmonic_centrality     0.132949
8                 square_clustering     0.127351
0  greedy_modularity_communities_id     0.120824
5              closeness_centrality     0.117354
7                        clustering     0.093332
2            betweenness_centrality     0.091710
3                 degree_centrality     0.054188
4                 k_path_centrality     0.045026
Atributos seleccionados:
Index(['greedy_modularity_communities_id', 'label_propagation_communities_id',
       'closeness_centrality', 'harmonic_centrality', 'square_clustering'],
      dtype='object')


/opt/homebrew/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [32]:
# Entrenar y evaluar el modelo con las mejores características
atributos_entrenamiento_selected = selector.fit_transform(atributos_entrenamiento, objetivo_entrenamiento)
atributos_prueba_selected = selector.transform(atributos_prueba)
mejor_modelo_RFC.fit(atributos_entrenamiento_selected, objetivo_entrenamiento)
predicciones_selected = mejor_modelo_RFC.predict(atributos_prueba_selected)
accuracy_selected = accuracy_score(objetivo_prueba, predicciones_selected)

print(f"Accuracy con mejores características: {accuracy_selected}")

Accuracy con mejores características: 0.5607476635514018


## <a name="gradient-boosting"></a> **Gradient Boosting**

In [33]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
modelo = GradientBoostingClassifier(random_state=42)

rejilla = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [1, 3, 5],
    'subsample': [0.8, 1.0]
}


# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(modelo, rejilla, cv=5, n_jobs=-1, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 1.0],
                         'max_depth': [1, 3, 5], 'n_estimators': [50, 100, 150],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy')

In [34]:
# Mejor modelo obtenido
mejor_modelo_GB = validaciones_cruzadas.best_estimator_
mejor_modelo_GB

GradientBoostingClassifier(max_depth=5, n_estimators=150, random_state=42,
                           subsample=0.8)

In [35]:
validaciones_cruzadas.best_params_

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 150, 'subsample': 0.8}

In [36]:
# Evaluar la importancia de los atributos
importancias_GB = mejor_modelo_GB.feature_importances_

# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_GB.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"Accuracy: {accuracy}")

# Crear un DataFrame para visualizar las importancias
if importancias_RFC is not None:
    importancias_atributos = pd.DataFrame({'Atributo': atributos_entrenamiento.columns, 'Importancia': importancias_GB})
    importancias_atributos = importancias_atributos.sort_values(by='Importancia', ascending=False)
    print(importancias_atributos)

# Seleccionar los atributos más importantes (umbral puede ser ajustado)
selector = SelectFromModel(mejor_modelo_GB, prefit=True, threshold='mean')
atributos_seleccionado = selector.transform(atributos_entrenamiento)

print("Atributos seleccionados:")
print(atributos_entrenamiento.columns[selector.get_support()])

Accuracy: 0.5947930574098799
                           Atributo  Importancia
1  label_propagation_communities_id     0.234752
6               harmonic_centrality     0.190495
0  greedy_modularity_communities_id     0.148058
8                 square_clustering     0.136682
2            betweenness_centrality     0.089435
7                        clustering     0.069796
5              closeness_centrality     0.059346
3                 degree_centrality     0.037368
4                 k_path_centrality     0.034068
Atributos seleccionados:
Index(['greedy_modularity_communities_id', 'label_propagation_communities_id',
       'harmonic_centrality', 'square_clustering'],
      dtype='object')


/opt/homebrew/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [37]:
# Entrenar y evaluar el modelo con las mejores características
atributos_entrenamiento_selected = selector.fit_transform(atributos_entrenamiento, objetivo_entrenamiento)
atributos_prueba_selected = selector.transform(atributos_prueba)
mejor_modelo_GB.fit(atributos_entrenamiento_selected, objetivo_entrenamiento)
predicciones_selected = mejor_modelo_GB.predict(atributos_prueba_selected)
accuracy_selected = accuracy_score(objetivo_prueba, predicciones_selected)

print(f"Accuracy con mejores características: {accuracy_selected}")

Accuracy con mejores características: 0.5531820204717401
